# Задание 1

Город разбит на квадраты и для каждого квадрата известно количество людей, живущих и работающих на территории этого квадрата. Эти данные находятся в файле `data.csv`. Например, на территории квадрата `85881` живет 101 человек и работает 28 человек.

Также город разбит на административные районы, их существенно меньше. Для каждого квадрата известно то, с какими районами он пересекается и по какой части своей площади. Эти данные находятся в файле `area2district.csv`. Например, примерно 38% площади квадрата `91422` составляет район `55`.

От Вас требуется предложить способ (и реализовать его) расчета количества людей, живущих и работающих в каждом административном районе соответственно. Можно считать, что внутри каждого квадрата люди распределены равномерно.

Требования:
   * В каждом районе должно жить и работать целое неотрицательное количество людей
   * Сумма проживающих и работающих жителей города не должна измениться

В данной задаче нет строго критерия оценивания, но важно сделать упор на скорость работы, в частности не рекомендуется пользоваться циклами `for`. 

In [1]:
import pandas as pd
import numpy as np

df_cell2adm = pd.read_csv('area2district.csv', sep=',')
df_data = pd.read_csv('data.csv', sep=',')

In [2]:
df_cell2adm

,areaid,districtid,percent
0,89012,478,1.000000
1,123048,55,0.984496
2,34536,7,1.000000
3,91422,55,0.380014
4,106142,55,0.912069
...,...,...,...
217176,1280,30,1.000000
217177,26567,139,0.023512
217178,92954,30,1.000000
217179,86425,30,0.074457


In [3]:
df_data

,areaid,home,job
0,144639,1,1
1,146472,1,1
2,18242,2,1
3,85881,101,28
4,27667,136,48
...,...,...,...
146291,81040,16,8
146292,187221,15,6
146293,115926,9,2
146294,43848,3,0


# My code

## Join & merge

In [4]:
def home_res(row):
    return row['percent'] * row['home']

In [5]:
def job_res(row):
    return row['percent'] * row['job']

In [6]:
%%time
df_result_pd_j = df_data.join(df_cell2adm.set_index('areaid'), on='areaid', how='inner')
df_result_pd_j['home_result'] = df_result_pd_j.apply(home_res, axis=1)
df_result_pd_j['job_result'] = df_result_pd_j.apply(job_res, axis=1)
df_result_pd_j

Wall time: 7.97 s


,areaid,home,job,districtid,percent,home_result,job_result
0,144639,1,1,466,1.0,1.0,1.0
1,146472,1,1,397,1.0,1.0,1.0
2,18242,2,1,285,1.0,2.0,1.0
3,85881,101,28,285,1.0,101.0,28.0
4,27667,136,48,285,1.0,136.0,48.0
...,...,...,...,...,...,...,...
146291,81040,16,8,297,1.0,16.0,8.0
146292,187221,15,6,472,1.0,15.0,6.0
146293,115926,9,2,47,1.0,9.0,2.0
146294,43848,3,0,5,1.0,3.0,0.0


In [7]:
%%time
# Объединяем датафреймы по 'areaid'
df_result_pd = df_data.merge(df_cell2adm, on='areaid', how='inner')
# Проходим функцииями построчно и вычисляем количество живущих и работающих в каждом районе
df_result_pd['home_result'] = df_result_pd.apply(home_res, axis=1)
df_result_pd['job_result'] = df_result_pd.apply(job_res, axis=1)

df_result_pd


Wall time: 8 s


,areaid,home,job,districtid,percent,home_result,job_result
0,144639,1,1,466,1.0,1.0,1.0
1,146472,1,1,397,1.0,1.0,1.0
2,18242,2,1,285,1.0,2.0,1.0
3,85881,101,28,285,1.0,101.0,28.0
4,27667,136,48,285,1.0,136.0,48.0
...,...,...,...,...,...,...,...
170126,81040,16,8,297,1.0,16.0,8.0
170127,187221,15,6,472,1.0,15.0,6.0
170128,115926,9,2,47,1.0,9.0,2.0
170129,43848,3,0,5,1.0,3.0,0.0


Метод .merge() оказался побыстрее, чем join()

In [8]:
# Убираем лишние
df_result_drop = df_result_pd.drop(['areaid','percent','home', 'job'],axis=1)
df_result_drop

,districtid,home_result,job_result
0,466,1.0,1.0
1,397,1.0,1.0
2,285,2.0,1.0
3,285,101.0,28.0
4,285,136.0,48.0
...,...,...,...
170126,297,16.0,8.0
170127,472,15.0,6.0
170128,47,9.0,2.0
170129,5,3.0,0.0


In [9]:
# Суммируем значения живущих и работающих в каждом районе
df_result_drop = df_result_drop.groupby('districtid').sum()
df_result_drop

,home_result,job_result
districtid,,
1,32625.162519,13566.430665
2,3716.605467,535.239616
3,46330.593462,18062.574715
4,3966.771421,1243.848413
5,4833.746494,391.347340
...,...,...
474,5557.526529,1934.038227
476,53653.821433,19837.551282
477,125351.117556,76267.900661


В том случае если округление проводить перед группировкой и суммированием по районам, погрешность получается больше.

Применяем метод округления .round() - он правильнее с математической точки зрения и показывает более точный результат чем .astype(int).

In [10]:
df_result_drop['home_result_rounded'] = df_result_drop['home_result'].round()
df_result_drop['job_result_rounded'] = df_result_drop['job_result'].round()
df_result_drop

,home_result,job_result,home_result_rounded,job_result_rounded
districtid,,,,
1,32625.162519,13566.430665,32625.0,13566.0
2,3716.605467,535.239616,3717.0,535.0
3,46330.593462,18062.574715,46331.0,18063.0
4,3966.771421,1243.848413,3967.0,1244.0
5,4833.746494,391.347340,4834.0,391.0
...,...,...,...,...
474,5557.526529,1934.038227,5558.0,1934.0
476,53653.821433,19837.551282,53654.0,19838.0
477,125351.117556,76267.900661,125351.0,76268.0


In [11]:
print('Число проживающих в городе после округления: ', df_result_drop['home_result_rounded'].sum())
print('Число проживающих в городе до округления: ', df_data['home'].sum())
print('Погрешность среди проживающих: ', df_result_drop['home_result_rounded'].sum() - df_data['home'].sum())
print('')
print('Число работающих в городе после округления: ', df_result_drop['job_result_rounded'].sum())
print('Число работающих в городе до округления: ', df_data['job'].sum())
print('Погрешность среди работающих: ', df_result_drop['job_result_rounded'].sum() - df_data['job'].sum())

Число проживающих в городе после округления:  19160905.0
Число проживающих в городе до округления:  19160892
Погрешность среди проживающих:  13.0

Число работающих в городе после округления:  9111367.0
Число работающих в городе до округления:  9111365
Погрешность среди работающих:  2.0


У нас образовалась небольшая остаточная погрешность. 

В задании есть требования, что сумма проживающих и работающих жителей города не должна измениться.

Можно поступить следующим образом: для минимизации влияния на данные можно исправить эту погрешность скорректировав данные в районах с наибольшим числом работающих и проживающих.

In [12]:
df_result_drop = df_result_drop.drop(['home_result', 'job_result'], axis=1)
df_result_drop

,home_result_rounded,job_result_rounded
districtid,,
1,32625.0,13566.0
2,3717.0,535.0
3,46331.0,18063.0
4,3967.0,1244.0
5,4834.0,391.0
...,...,...
474,5558.0,1934.0
476,53654.0,19838.0
477,125351.0,76268.0


In [13]:
# Сортируем данные по числу проживающих в районе
df_result_drop_sort_home = df_result_drop.sort_values('home_result_rounded', ascending=False)
df_result_drop_sort_home.head(30)

,home_result_rounded,job_result_rounded
districtid,,
119,549428.0,192817.0
121,423096.0,195672.0
361,395497.0,152536.0
165,323645.0,155103.0
23,242258.0,83562.0
30,237939.0,105724.0
276,224964.0,87767.0
185,215323.0,83453.0
286,201722.0,56159.0


In [14]:
def plus_home(row):
    return row['home_result_rounded'] - 1

In [15]:
# Записываем погрешность среди проживающих в переменную
ind_x = int(df_result_drop['home_result_rounded'].sum() - df_data['home'].sum())

# Разделяем датафрейм на 2. В первом датафрейме количесто строк равно погрешности.
x_df_result_drop_sort_home = df_result_drop_sort_home[['home_result_rounded', 'job_result_rounded']][:ind_x]
y_df_result_drop_sort_home = df_result_drop_sort_home[['home_result_rounded', 'job_result_rounded']][ind_x:]

# Построчно отнимаем по единице из каждого значения в колонке с проживающими в первом датафрейме
x_df_result_drop_sort_home['home_result_rounded'] = x_df_result_drop_sort_home.apply(plus_home, axis=1)

# Объединяем
df_result_drop_sort_home = pd.concat([x_df_result_drop_sort_home, y_df_result_drop_sort_home])


Аналогично проделываем с погрешностью среди работающих.

In [16]:
df_result_drop_sort_job = df_result_drop_sort_home.sort_values('job_result_rounded', ascending=False)

In [17]:
def plus_job(row):
    return row['job_result_rounded'] - 1

In [18]:
ind_x = int(df_result_drop['job_result_rounded'].sum() - df_data['job'].sum())

x_df_result_drop_sort_job = df_result_drop_sort_job[['home_result_rounded', 'job_result_rounded']][:ind_x]
y_df_result_drop_sort_job = df_result_drop_sort_job[['home_result_rounded', 'job_result_rounded']][ind_x:]

x_df_result_drop_sort_job['job_result_rounded'] = x_df_result_drop_sort_job.apply(plus_job, axis=1)

df_result_drop_sort_job = pd.concat([x_df_result_drop_sort_job, y_df_result_drop_sort_job])

In [19]:
df_result_drop_end = df_result_drop_sort_job

In [20]:
print('Число проживающих в городе после округления: ', df_result_drop_end['home_result_rounded'].sum())
print('Число проживающих в городе до округления: ', df_data['home'].sum())
print('Погрешность среди проживающих: ', df_result_drop_end['home_result_rounded'].sum() - df_data['home'].sum())
print('')
print('Число работающих в городе после округления: ', df_result_drop_end['job_result_rounded'].sum())
print('Число работающих в городе до округления: ', df_data['job'].sum())
print('Погрешность среди работающих: ', df_result_drop_end['job_result_rounded'].sum() - df_data['job'].sum())

Число проживающих в городе после округления:  19160892.0
Число проживающих в городе до округления:  19160892
Погрешность среди проживающих:  0.0

Число работающих в городе после округления:  9111365.0
Число работающих в городе до округления:  9111365
Погрешность среди работающих:  0.0


Теперь количество совпадает

### Поставленная задача выполнена

Дополнительно попробуем пройтись циклом и посмотреть затраты по времени.

In [21]:
df_result = pd.DataFrame(df_cell2adm.districtid.unique())
df_result.columns = ['districtid']
df_result['home'] = 0
df_result['job'] = 0
df_result 

,districtid,home,job
0,478,0,0
1,55,0,0
2,7,0,0
3,407,0,0
4,466,0,0
...,...,...,...
438,96,0,0
439,308,0,0
440,264,0,0
441,291,0,0


Попробуем пройтись циклом и заполнить колонку 'home'.

In [22]:
def result_home():
    df_ind = df_data.index.max()+1
    ind_df_data = df_data.index.max()
    for x in range(0, df_ind):
        temporarily = df_cell2adm[df_cell2adm['areaid'] == df_data['areaid'][x]]
        temporarily.reset_index(drop=True, inplace=True)
        temp_ind = temporarily.index.max()+1

        def inside_home():
            for y in range(0, temp_ind):
                index = df_result[df_result['districtid'] == temporarily['districtid'][y]].index
                home_percent = temporarily['percent'][y]
                home_result = round(df_data['home'][x] * home_percent)
                df_result.loc[index, 'home'] = df_result['home'][index] + home_result
        inside_home()

In [23]:
%%time
result_home()

Wall time: 9min 48s


In [24]:
df_result

,districtid,home,job
0,478,12270,0
1,55,14822,0
2,7,14881,0
3,407,4952,0
4,466,6659,0
...,...,...,...
438,96,77872,0
439,308,83495,0
440,264,93076,0
441,291,68225,0


Циклом данная задача решается неэффективно.